In [7]:
# -*- coding: utf-8 -*-

"""
批量改名
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
Ghent University
Haoyu.Wang@ugent.be
"""

from tqdm import tqdm
from osgeo import gdal,ogr
import fnmatch
from PIL import Image
from numpy import *
import cv2
import matplotlib.pyplot as plt
import os

In [8]:
def write_img(out_path, im_proj, im_geotrans, im_data):
    """output img

    Args:
        out_path: Output path
        im_proj: Affine transformation parameters
        im_geotrans: spatial reference
        im_data: Output image data

    """
    # identify data type 
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    # calculate number of bands
    if len(im_data.shape) > 2:  
        im_bands, im_height, im_width = im_data.shape
    else:  
        im_bands, (im_height, im_width) = 1, im_data.shape

    # create new img
    driver = gdal.GetDriverByName("GTiff")
    new_dataset = driver.Create(
        out_path, im_width, im_height, im_bands, datatype)
    new_dataset.SetGeoTransform(im_geotrans)
    new_dataset.SetProjection(im_proj)
    if im_bands == 1:
        new_dataset.GetRasterBand(1).WriteArray(im_data.squeeze())
    else:
        for i in range(im_bands):
            new_dataset.GetRasterBand(i + 1).WriteArray(im_data[i])

    del new_dataset

def read_img(sr_img):
    """read img

    Args:
        sr_img: The full path of the original image

    """
    im_dataset = gdal.Open(sr_img)
    if im_dataset == None:
        print('open sr_img false')
        sys.exit(1)
    im_geotrans = im_dataset.GetGeoTransform()
    im_proj = im_dataset.GetProjection()
    im_width = im_dataset.RasterXSize
    im_height = im_dataset.RasterYSize
    im_bands = im_dataset.RasterCount 
    im_data = im_dataset.ReadAsArray(0, 0, im_width, im_height)
    del im_dataset

    return im_data, im_proj, im_geotrans, im_height, im_width, im_bands

In [9]:
img_path = r'F:\project_populus_GF2_and_UAV\0-clip_polygon_img\16-UAV-321-resample-2-brightness_adjust'
output_path = r'F:\project_populus_GF2_and_UAV\0-clip_polygon_img\17-UAV-321-resample-2-brightness_adjust-clip1000px'
suffix = ''  # 自定义后缀
datatype = 'png'


listpic = fnmatch.filter(os.listdir(img_path), '*.tif')

for img_name in listpic:
    img_full_path = img_path + '/' + img_name
    im_data, im_proj, im_geotrans = read_img(img_full_path)[:3]

    output_full_path = output_path + '/' + img_name[:-4] + suffix + '.' + datatype
    write_img(output_full_path, im_proj, im_geotrans, im_data)